### InMemoryChatMessageHistory
Контекст диалога будет храниться в оперативной памяти. После переапуска обнулится
Для этого создается обертка RunnableWithMessageHistory

При таком раскладе в конфигурацию добавляется **session ID** и (если нужно) **user ID**. Так можно хранить историю разных
диалогов разных людей

Для подгрузки в память можно юзать такие методы:
- add_user_message() - для сообщений юзера
- add_ai_message() - для сообщений AI
- add_message()  - для любых сообщений
- aadd_message() - для асинхронного добавления

In [1]:
import os
from getpass import getpass
from langchain_openai import ChatOpenAI

course_api_key = getpass(prompt='Введите API-ключ полученный в боте')

# инициализируем языковую модель
llm = ChatOpenAI(api_key=course_api_key, model='gpt-4o-mini', 
                 base_url="https://aleron-llm.neuraldeep.tech/")

In [2]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

#Создаем словарь который будет маппить session_id с историей
store = {}

#Функция возвращающая историю диалога по session_id
def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [3]:
#Создаем конфиг в который будем передавать session_id
config = {"configurable" : {"session_id": "1"}}

llm_with_history = RunnableWithMessageHistory(llm,get_session_history)

#Теперь при вызове надо добавлять экземпляр config с нужной session_id
answer = llm_with_history.invoke("Привет GPT! Меня зовут Виктор, как дела?",config=config).content

In [ ]:
print(answer) #Привет, Виктор! У меня всё хорошо, спасибо. Как дела у тебя? Чем могу помочь?

print(store['1'])
# Human: Привет GPT! Меня зовут Виктор, как дела?
# AI: Привет, Виктор! У меня всё хорошо, спасибо. Как дела у тебя? Чем могу помочь?

Привет, Виктор! У меня всё хорошо, спасибо. Как дела у тебя? Чем могу помочь?
Human: Привет GPT! Меня зовут Виктор, как дела?
AI: Привет, Виктор! У меня всё хорошо, спасибо. Как дела у тебя? Чем могу помочь?


In [6]:
#Теперь при вызове надо добавлять экземпляр config с нужной session_id
answer = llm_with_history.invoke("Как меня зовут?",config=config).content

In [ ]:
print(answer) #Тебя зовут Виктор. Как я могу помочь тебе сегодня?

#Модель запомнила контекст и всё находится в сессии

Тебя зовут Виктор. Как я могу помочь тебе сегодня?


In [8]:
# Очистить историю диалога сессии можно методом clear()
get_session_history('1').clear()
store

{'1': InMemoryChatMessageHistory(messages=[])}

In [ ]:
#Добавим сообщения в сессию
get_session_history('1').clear()
get_session_history('1').add_user_message('Hello, World')
get_session_history('1').add_ai_message('Hi')

print(store['1'])

# Human: Hello, World
# AI: Hi

Human: Hello, World
AI: Hi


### ChatPromptTemplate
Встраиваем историю в промпт

In [11]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

#Создаем шаблон для ассистента
prompt = ChatPromptTemplate.from_messages([
    ('system', "Ты полезный ассистент, отлично разбирающийся в {topic}")
    ,MessagesPlaceholder(variable_name='chat_history')  #Заглушка куда будем поставлять историю диалога
    ,('human', '{question}')
    ])


In [12]:
#Собираем цепочку
chain = prompt | llm

#Добавляем сохранение истории взаимодействий
chain_with_history = RunnableWithMessageHistory(chain
                                                ,get_session_history
                                                ,input_messages_key="question"
                                                ,history_messages_key='chat_history')

#Выполняем запрос
answer = chain_with_history.invoke({"topic": "математика", "question": "Чему равен синус 30?"}
                          ,config= {"configurable" : {"session_id": "2"}}).content


In [ ]:
print(answer) #Синус 30 градусов равен \( \frac{1}{2} \) или 0.5.

Синус 30 градусов равен \( \frac{1}{2} \) или 0.5.


In [14]:
#А теперь задаем второй вопрос, по контексту уже модель понимает что мы ходим
answer = chain_with_history.invoke(
    {"topic": "математика", "question": "А чему равен косинус?"},
    config={"configurable": {"session_id": "2"}}).content

In [ ]:
print(answer) #Косинус 30 градусов равен \( \frac{\sqrt{3}}{2} \) или примерно 0.866.

#Модель понимает контекст

Косинус 30 градусов равен \( \frac{\sqrt{3}}{2} \) или примерно 0.866.


### Минусы таких подходов

Недостатки InMemoryChatMessageHistory:

- История хранится в оперативной памяти. При интенсивных диалогах и большом количестве пользователей память может закончиться
- При перезагрузке или сбое сервера - вся история исчезнет
- Чем длиннее история, тем больше токенов придется подавать на вход модели
- В случае платных моделей, это будет накладно по финансам
- Контекстное окно моделей тоже ограничено по количеству токенов

Для продакшен решений можно рассмотреть более практичные форматы хранения истории в Langchain, например, релизованы:

- FileChatMessageHistory - сохраняет историю взаимодействия сообщений в файл.
- RedisChatMessageHistory - сохраняет историю сообщений чата в базе данных Redis.
- SQLChatMessageHistory - сохраняет историю сообщений чата в базе данных SQL.
- MongoDBChatMessageHistory - в базе данных Mongo
и многие другие - https://python.langchain.com/api_reference/community/chat_message_histories.html

Например, это могло бы выглядеть так:

engine = sqlalchemy.create_engine("sqlite:///database/chat_history.db")

SQLChatMessageHistory(session_id=session_id, connection=engine)